# [MICRO-452:] Project Report



<p><b>Authors:</b> &nbsp;&emsp;&emsp;&emsp;&emsp;&emsp;Antoine Rol, Ruben Jungius, Hugo Masson, Jadd<br>
<b>Supervisors:</b> &nbsp;&emsp;&emsp;&emsp;Prof. Francesco Mondada<br>
<b>Due date:</b>  &nbsp;&nbsp;&nbsp;&emsp;&emsp;&emsp;&emsp;December 7th, 2023</p>
<b>Presentation date:</b> &nbsp;&nbsp;December 14th, 2023</p>

<h1><center> Thymhiker  </center></h1>
<img src="" style="width: 600px;"> 

<h1>Table of Contents<span class="tocSkip"></span></h1><br>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1.&nbsp;&nbsp;</span>Introduction</a></span></li></ul><ul class="toc-item"><li><a href="#Project-summary-,-hardware-and-choices" data-toc-modified-id="Project-summary-,-hardware-and-choices-2"><span class="toc-item-num">2.&nbsp;&nbsp;</span>Project summary, hardware and choices</a></li></ul><ul class="toc-item"><li><span><a href="#Vision" data-toc-modified-id="Vision-3"><span class="toc-item-num">3.&nbsp;&nbsp;</span>Vision</a></span></li></ul><ul class="toc-item"><li><a href="#Global-navigation" data-toc-modified-id="Global-navigation-4"><span class="toc-item-num">4.&nbsp;&nbsp;</span>Global navigation</a></li></ul><ul class="toc-item"><li><a href="#Local-navigation" data-toc-modified-id="Local-navigation-5"><span class="toc-item-num">5.&nbsp;&nbsp;</span>Local navigation</a></li></ul><ul class="toc-item"><li><a href="#Filtering" data-toc-modified-id="Filtering-6"><span class="toc-item-num">6.&nbsp;&nbsp;</span>Filtering</a></li></ul><ul class="toc-item"><li><a href="#Motion-control" data-toc-modified-id="Motion-control-7"><span class="toc-item-num">7.&nbsp;&nbsp;</span>Motion control</a></li></ul><ul class="toc-item"><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-8"><span class="toc-item-num">8.&nbsp;&nbsp;</span>Conclusion</a></span></li></ul><ul class="toc-item"><li><a href="#Main" data-toc-modified-id="Main-9"><span class="toc-item-num">9.&nbsp;&nbsp;</span>Main</a></ul> 

# I. Introduction
<a class="anchor" id="Introduction"></a>
<p style='text-align: justify;'> 

# II. Project summary, hardware and choices
<a class="anchor" id="Project-summary-,-hardware-and-choices"></a>
### Project summary

# III. Vision
<a class="anchor" id="Vision"></a>

# IV. Global navigation
<a class="anchor" id="Global-navigation"></a>

# V. Local Navigation
<a class="anchor" id="Local-navigation"></a>

# VI. Filtering
<a class="anchor" id="Filtering"></a>

# VII. Motion Control
<a class="anchor" id="Motion-control"></a>

# VIII . Conclusion
<a class="anchor" id="Conclusion"></a>